<a href="https://colab.research.google.com/github/arlakhan/Deep-Fake-Detection/blob/main/Video_Fake_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split

# -------------------------------
# Parameters
# -------------------------------
IMG_SIZE = 128
NUM_FRAMES = 3
VIDEO_FOLDER = 'dataset'  # Structure: dataset/real/*.mp4 and dataset/fake/*.mp4

# -------------------------------
# Extract frames from a video
# -------------------------------
def extract_frames(video_path, num_frames=NUM_FRAMES):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ Error: Could not open video {video_path}")
        return None

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_ids = np.linspace(0, total_frames - 1, num_frames, dtype=int)

    frames = []
    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        if i in frame_ids:
            frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
            frames.append(frame)

    cap.release()
    if len(frames) < num_frames:
        print(f"⚠️ Warning: Only {len(frames)} frames extracted from {video_path}")
        return None
    return np.array(frames)

# -------------------------------
# Extract features using MobileNetV2
# -------------------------------
def extract_features(frames, feature_extractor):
    frames = tf.keras.applications.mobilenet_v2.preprocess_input(frames)
    features = feature_extractor.predict(frames, verbose=0)
    pooled = tf.keras.layers.GlobalAveragePooling2D()(features)
    flattened = tf.reshape(pooled, [-1])
    return flattened.numpy()

# -------------------------------
# Prepare dataset
# -------------------------------
def prepare_dataset(video_folder, feature_extractor, max_videos=10):
    X, y = [], []
    for label, folder in enumerate(['real', 'fake']):
        folder_path = os.path.join(video_folder, folder)
        if not os.path.exists(folder_path):
            print(f"⚠️ Warning: Folder {folder_path} not found.")
            continue

        video_files = [f for f in os.listdir(folder_path) if f.endswith('.mp4')][:max_videos]
        for video_file in video_files:
            video_path = os.path.join(folder_path, video_file)
            print(f"📹 Processing: {video_path}")
            frames = extract_frames(video_path, NUM_FRAMES)
            if frames is not None:
                features = extract_features(frames, feature_extractor)
                X.append(features)
                y.append(label)
    return np.array(X), np.array(y)

# -------------------------------
# Build simple classification model
# -------------------------------
def build_model(input_shape):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# -------------------------------
# Predict on a single video
# -------------------------------
def predict_video(video_path, trained_model, feature_extractor, num_frames=NUM_FRAMES):
    frames = extract_frames(video_path, num_frames)
    if frames is None or len(frames) < num_frames:
        print("❌ Error: Not enough frames for prediction.")
        return None

    features = extract_features(frames, feature_extractor)
    prediction = trained_model.predict(np.expand_dims(features, axis=0), verbose=0)[0][0]

    label = "Real Video ✅" if prediction < 0.5 else "Fake Video ❌"
    confidence = 1 - prediction if prediction < 0.5 else prediction

    print(f"\n📊 Prediction Result for: {video_path}")
    print(f"🔎 Status: {label}")
    print(f"📈 Confidence Score: {confidence:.4f}")

    return label, confidence

# -------------------------------
# Main execution
# -------------------------------
if __name__ == '__main__':
    print("🚀 Loading MobileNetV2 model...")
    mobilenet_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

    print("📁 Preparing dataset...")
    X, y = prepare_dataset(VIDEO_FOLDER, mobilenet_model, max_videos=20)

    if len(X) == 0:
        print("❌ No features extracted. Please check your dataset structure.")
    else:
        print("🧠 Training classifier...")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        model = build_model(input_shape=(X_train.shape[1],))
        model.fit(X_train, y_train, epochs=10, batch_size=4, validation_data=(X_test, y_test))

        print("💾 Saving model to 'video_fake_detection_model.h5'")
        model.save('video_fake_detection_model.h5')

        # Example prediction
        test_video_path = 'dataset/fake/0003_fake.mp4'  # Replace with your test video path
        #test_video_path = 'dataset/real/0001.mp4'  # Replace with your test video path
        predict_video(test_video_path, model, mobilenet_model)


🚀 Loading MobileNetV2 model...
📁 Preparing dataset...
📹 Processing: dataset/real/0041.mp4
📹 Processing: dataset/real/0032.mp4
📹 Processing: dataset/real/0024.mp4
📹 Processing: dataset/real/0027.mp4
📹 Processing: dataset/real/0004.mp4
📹 Processing: dataset/real/0016.mp4
📹 Processing: dataset/real/0013.mp4
📹 Processing: dataset/real/0047.mp4
📹 Processing: dataset/real/0009.mp4
📹 Processing: dataset/real/0040.mp4
📹 Processing: dataset/real/0001.mp4
📹 Processing: dataset/real/0010.mp4
📹 Processing: dataset/real/0025.mp4
📹 Processing: dataset/real/0031.mp4
📹 Processing: dataset/real/0042.mp4
📹 Processing: dataset/real/0017.mp4
📹 Processing: dataset/real/0008.mp4
📹 Processing: dataset/real/0033.mp4
📹 Processing: dataset/real/0036.mp4
📹 Processing: dataset/real/0035.mp4
📹 Processing: dataset/fake/0038_fake.mp4
📹 Processing: dataset/fake/0003_fake.mp4
📹 Processing: dataset/fake/0010_fake.mp4
📹 Processing: dataset/fake/0005_fake.mp4
📹 Processing: dataset/fake/0000_fake.mp4
📹 Processing: dataset

💾 Saving model to 'video_fake_detection_model.h5'

📊 Prediction Result for: dataset/fake/0003_fake.mp4
🔎 Status: Fake Video ❌
📈 Confidence Score: 0.9660
